# Quantlib pricer code

## Qauntlib pricer

In [1]:
import QuantLib as ql
import math

def zero_curve_from_csv(valuation_date: ql.Date, rows, day_count=ql.Actual365Fixed(), cal=ql.UnitedStates(ql.UnitedStates.NYSE)):
    """
    rows: iterable of (T_years, r_cont_zero). Builds a ZeroCurve dated from valuation_date.
    """
    dates = [valuation_date] + [valuation_date + ql.Period(int(round(T*365)), ql.Days) for T, _ in rows]
    rates = [rows[0][1]] + [float(r) for _, r in rows]  # pad front; QL expects >= 2 nodes
    return ql.YieldTermStructureHandle(ql.ZeroCurve(dates, rates, day_count, cal))

class QLAmericanPricer:
    def __init__(self, valuation_date: ql.Date, r_curve: ql.YieldTermStructureHandle,
                 init_vol: float = 0.20, cal=ql.UnitedStates(ql.UnitedStates.NYSE), dc=ql.Actual365Fixed(),
                 t_grid: int = 400, x_grid: int = 200):
        ql.Settings.instance().evaluationDate = valuation_date
        self.val_date = valuation_date
        self.cal, self.dc = cal, dc
        self.t_grid, self.x_grid = t_grid, x_grid

        # live quotes you can bump
        self.spot_q = ql.SimpleQuote(0.0)
        self.vol_q  = ql.SimpleQuote(max(1e-8, init_vol))

        # handles
        self.u   = ql.QuoteHandle(self.spot_q)
        self.vts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(valuation_date, cal, ql.QuoteHandle(self.vol_q), dc))
        self.rts = r_curve
        self.dts = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, 0.0, dc, ql.Continuous))  # cont. q=0; use cash divs explicitly

        # stochastic process reused across prices
        self.process = ql.BlackScholesMertonProcess(self.u, self.dts, self.rts, self.vts)

    def _div_schedule(self, dividends):
        sched = []
        if not dividends: return sched
        for d, amt in dividends:
            dq = d if isinstance(d, ql.Date) else ql.Date(d.day, d.month, d.year)
            if dq > self.val_date:
                sched.append(ql.FixedDividend(float(amt), dq))
        return sched

    def price(self, S, K, maturity: ql.Date, cp: str,
            vol: float | None = None, dividends: list[tuple] | None = None,
            want_greeks: bool = True, vega_eps: float = 1e-4,
            borrow: float | None = None):          # <= NEW

        # live quotes
        self.spot_q.setValue(float(S))
        if vol is not None:
            self.vol_q.setValue(max(1e-8, float(vol)))

        payoff   = ql.PlainVanillaPayoff(ql.Option.Call if cp.upper().startswith('C') else ql.Option.Put, float(K))
        exercise = ql.AmericanExercise(self.val_date, maturity)
        option   = ql.VanillaOption(payoff, exercise)

        # --- per-option borrow: build a LOCAL dividend-yield term structure ---
        # Convention: borrow>0 lowers forward like a dividend yield q (i.e., reduces C-P).
        if borrow is not None:
            dts_local = ql.YieldTermStructureHandle(
                ql.FlatForward(self.val_date, float(borrow), self.dc, ql.Continuous)
            )
            process_local = ql.BlackScholesMertonProcess(self.u, dts_local, self.rts, self.vts)
        else:
            dts_local = self.dts  # whatever is set globally (often 0)
            process_local = self.process

        divs = self._div_schedule(dividends or [])

        # --- Engine with explicit cash dividends (preferred path) ---
        engine = None
        try:
            engine = ql.FdBlackScholesVanillaEngine(process_local, divs, self.t_grid, self.x_grid)
        except TypeError:
            # Fallback: escrow PV of discrete divs out of spot if engine signature without schedule is hit
            pv = sum(d.amount() * self.rts.discount(d.date()) for d in divs)
            self.spot_q.setValue(float(S) - pv)
            engine = ql.FdBlackScholesVanillaEngine(process_local, self.t_grid, self.x_grid)

        option.setPricingEngine(engine)

        out = {"theo": option.NPV()}
        if not want_greeks:
            # restore spot if escrow used
            self.spot_q.setValue(float(S))
            return out

        # Greeks (engine-provided if available)
        for g in ("delta", "gamma", "theta"):
            try:
                out[g] = getattr(option, g)()
            except RuntimeError:
                out[g] = None

        # Vega (fallback numeric if engine lacks it)
        try:
            out["vega"] = option.vega()
        except RuntimeError:
            v0 = self.vol_q.value()
            self.vol_q.setValue(v0 + vega_eps)
            p_up = ql.VanillaOption(payoff, exercise); p_up.setPricingEngine(engine); upv = p_up.NPV()
            self.vol_q.setValue(v0 - vega_eps)
            p_dn = ql.VanillaOption(payoff, exercise); p_dn.setPricingEngine(engine); dnv = p_dn.NPV()
            self.vol_q.setValue(v0)
            out["vega"] = ((upv - dnv) / (2*vega_eps)) * 0.01  # per 1 vol point

        # restore spot if escrow path
        self.spot_q.setValue(float(S))
        return out

## quantlib dataframe pricer

In [2]:
# pip install QuantLib-Python pandas numpy
import pandas as pd, numpy as np
from datetime import datetime

# ---------- helpers ----------
def _to_qldate(s: str) -> ql.Date:
    # supports 'm/d/yy' or 'm/d/yyyy'
    fmt = '%m/%d/%y' if len(s.rsplit('/',1)[-1])<=2 else '%m/%d/%Y'
    d = datetime.strptime(s, fmt)
    return ql.Date(d.day, d.month, d.year)

def _flat_curve(valuation_date: str | ql.Date, r_cont: float,
                dc=ql.Actual365Fixed()) -> ql.YieldTermStructureHandle:
    vd = _to_qldate(valuation_date) if isinstance(valuation_date, str) else valuation_date
    return ql.YieldTermStructureHandle(ql.FlatForward(vd, r_cont, dc, ql.Continuous))

def _div_list(dividends_df: pd.DataFrame) -> list[tuple]:
    # returns [(ql.Date, amount), ...] sorted
    out = []
    for d, a in zip(dividends_df['Date'], dividends_df['Amount']):
        dq = _to_qldate(str(d)) if not isinstance(d, ql.Date) else d
        out.append((dq, float(a)))
    out.sort(key=lambda x: (int(x[0].serialNumber()),))
    return out

# ---------- wrapper ----------
def price_table_with_divs_ql(options_df: pd.DataFrame,
                             dividends_df: pd.DataFrame,
                             valuation_date='10/17/2025',
                             r=0.0308,
                             init_vol=0.20,
                             t_grid=400, x_grid=200,
                             want_greeks=True,
                             compute_combo=True):
    """
    Uses QLAmericanPricer.price(...) for each row in options_df.
    Expects columns: ['Expiration','Strike','Spot','CP','Vol'] (Vol may be '27.5%' or 0.275)
                     Optional: 'mult'
    Discrete cash dividends provided in dividends_df: columns ['Date','Amount'].
    """
    # Build curve & pricer
    val_qldate = _to_qldate(valuation_date)
    r_curve = _flat_curve(val_qldate, r)
    pricer = QLAmericanPricer(val_qldate, r_curve, init_vol=init_vol, t_grid=t_grid, x_grid=x_grid)

    # Normalize dividends to list of (ql.Date, amt)
    divs = _div_list(dividends_df)

    rows = []
    for i, row in options_df.iterrows():
        # Parse row fields
        S  = float(row['Spot'])
        K  = float(row['Strike'])
        cp = str(row['CP']).upper()[0]
        exp = row['Expiration']
        mat = _to_qldate(str(exp)) if not isinstance(exp, ql.Date) else exp
        borrow = float(row['Borrow'])

        vol_raw = row['Vol']
        if isinstance(vol_raw, str) and vol_raw.strip().endswith('%'):
            vol = float(vol_raw.strip('%'))/100.0
        else:
            vol = float(vol_raw)

        # Price with your engine
        res = pricer.price(S=S, K=K, maturity=mat, cp=cp, vol=vol, dividends=divs,
                           want_greeks=want_greeks,borrow=borrow)

        rows.append({
            **row.to_dict(),
            'Theo': res['theo'],
            'Delta': res.get('delta'),
            'Gamma': res.get('gamma'),
            'Theta': res.get('theta'),
            'Vega':  res.get('vega')
        })

    priced = pd.DataFrame(rows)

    if compute_combo:
        # C - P per (Expiration, Strike); assumes both legs exist in table
        grp_cols = ['Expiration', 'Strike']
        # pivot to wide then compute combo
        wide = priced.pivot_table(index=grp_cols, columns='CP', values='Theo', aggfunc='first')
        wide['Combo_C_minus_P'] = wide.get('C', np.nan) - wide.get('P', np.nan)
        priced = priced.merge(wide[['Combo_C_minus_P']].reset_index(), on=grp_cols, how='left')

    return priced


# Stand alone PDE pricer

In [6]:
# pip install scipy
import numpy as np
from scipy.linalg import solve_banded

def american_pde_price_banded(
    S0, K, T, sigma,
    r_func, q_func=lambda t: 0.0,
    dividends=None,                 # [(t_years, cash), ...]
    cp='C',
    NS=400, NT=800,                 # you can often use NT ~ NS with Rannacher
    theta=0.5,                      # 0.5 = Crank–Nicolson
    rannacher=2,                    # first 2 steps implicit Euler to smooth payoff kink
    Smax=None,
    return_grid=False
):
    """FDM PDE with discrete cash dividends, early exercise, banded solver."""
    dividends = sorted(dividends or [], key=lambda x: x[0])

    # spot grid
    if Smax is None:
        D_sum = sum(d for _, d in dividends)
        base = max(S0, K)
        Smax = max(4*base, 2*base + 4*D_sum)
    S = np.linspace(0.0, Smax, NS+1).astype(np.float64)
    dS = S[1] - S[0]

    # payoff at maturity
    V = (S - K).clip(min=0.0) if cp.upper().startswith('C') else (K - S).clip(min=0.0)

    # time grid (include dividend times exactly)
    times = list(np.linspace(0.0, T, NT+1))
    for t, _ in dividends:
        if 0 < t < T: times.append(t)
    times = np.array(sorted(set(times)), dtype=np.float64)  # ascending
    div_amt = {t: 0.0 for t, _ in dividends if 0 < t <= T}
    for t, a in dividends:
        if 0 < t <= T: div_amt[t] = div_amt.get(t, 0.0) + float(a)
    div_times = set(div_amt.keys())

    # optional full grid storage
    V_grid = None
    if return_grid:
        V_grid = np.empty((len(times), NS+1), dtype=np.float64)
        V_grid[-1] = V

    # prealloc work arrays
    N = NS - 1                         # number of interior nodes
    Si = S[1:-1]
    lower = np.empty(N, dtype=np.float64)
    diag  = np.empty(N, dtype=np.float64)
    upper = np.empty(N, dtype=np.float64)
    rhs   = np.empty(N, dtype=np.float64)
    # banded matrix container (3, N): [upper; diag; lower]
    ab = np.zeros((3, N), dtype=np.float64)

    # backward time stepping
    steps_done = 0
    for j in range(len(times)-1, 0, -1):
        t_cur, t_prev = times[j], times[j-1]
        dt = t_cur - t_prev
        th = 1.0 if steps_done < rannacher else theta
        t_mid = t_prev + th*dt

        r = float(r_func(t_mid)); q = float(q_func(t_mid))
        sig2 = float(sigma)*float(sigma)

        A = 0.5 * sig2 * (Si**2) / (dS**2)
        B = (r - q) * Si / (2.0*dS)

        # LHS (theta part)
        lower[:] = -th * dt * (A - B)   # length N
        diag[:]  =  1.0 + th * dt * (2.0*A + r)
        upper[:] = -th * dt * (A + B)

        # RHS ((1-theta) part)
        lower_r = (1.0 - th) * dt * (A - B)
        diag_r  = 1.0 - (1.0 - th) * dt * (2.0*A + r)
        upper_r = (1.0 - th) * dt * (A + B)

        # boundaries
        if cp.upper().startswith('C'):
            V0, VN = 0.0, S[-1]-K
        else:
            V0, VN = K, 0.0

        rhs[:] = lower_r * V[:-2] + diag_r * V[1:-1] + upper_r * V[2:]
        rhs[0]  -= lower[0] * V0
        rhs[-1] -= upper[-1] * VN

        # build banded tri-diagonal:
        # ab[0,1:] = upper[:-1], ab[1,:] = diag, ab[2,:-1] = lower[1:]
        ab[0, 1:] = upper[:-1]
        ab[1, :]  = diag
        ab[2, :-1]= lower[1:]
        # solve
        V_inner = solve_banded((1, 1), ab, rhs, overwrite_ab=True, overwrite_b=True, check_finite=False)

        V[0], V[-1] = V0, VN
        V[1:-1] = V_inner

        # early exercise
        if cp.upper().startswith('C'):
            np.maximum(V, S - K, out=V)
        else:
            np.maximum(V, K - S, out=V)

        # discrete dividend jump at t_prev
        if t_prev in div_times:
            D = div_amt[t_prev]
            S_shift = np.clip(S - D, 0.0, S[-1])
            V[:] = np.interp(S_shift, S, V)
            # re-enforce exercise right before ex-div
            if cp.upper().startswith('C'):
                np.maximum(V, S - K, out=V)
            else:
                np.maximum(V, K - S, out=V)

        if return_grid:
            V_grid[j-1] = V

        steps_done += 1

    price = float(np.interp(S0, S, V))
    debug = {"S_grid": S, "times": times}
    if return_grid:
        debug["V_grid"] = V_grid
    else:
        debug["V0"] = V
    return price, debug

In [7]:
def american_pde_price_and_greeks(
    S0, K, T, sigma,
    r_func, q_func=lambda t: 0.0,
    dividends=None,
    cp='C',
    NS=400, NT=800,
    theta=0.5,            # CN scheme weight (not the Greek)
    rannacher=2,
    Smax=None,
    return_grid=False,
    vol_bump=0.01,        # 1 vol point (absolute, i.e., 0.01 = 1%)
    rate_bump=1e-4        # 1 basis point
):
    # --- base run with grid so we can read theta from the first time step ---
    p0, dbg0 = american_pde_price_banded(
        S0, K, T, sigma, r_func, q_func, dividends, cp,
        NS, NT, theta, rannacher, Smax, return_grid=True
    )
    Sg      = dbg0["S_grid"]
    times   = dbg0["times"]
    V_grid  = dbg0["V_grid"]         # time x spot
    Vt0     = V_grid[0]              # t=0 slice

    # --- delta/gamma from t=0 slice (your existing approach) ---
    def _delta_gamma_from_grid(S, V, S0, h=None):
        S = np.asarray(S); V = np.asarray(V)
        dS = S[1]-S[0]; h = max(dS, 1e-4*max(S0,1.0)) if h is None else h
        Sm, Sp = max(S[0], S0-h), min(S[-1], S0+h)
        Vm, V0, Vp = np.interp([Sm, S0, Sp], S, V)
        delta = (Vp - Vm) / (Sp - Sm)
        h_eff = max(1e-12, 0.5*(Sp-Sm))
        gamma = (Vp - 2*V0 + Vm) / (h_eff**2)
        return float(delta), float(gamma)

    delta, gamma = _delta_gamma_from_grid(Sg, Vt0, S0)

    # helper: price at S0 from a time-slice
    def _p_from_slice(Vslice):
        return float(np.interp(S0, Sg, Vslice))

    # --- theta (calendar dV/dt at t=0) from first time step on the grid ---
    if len(times) > 1:
        dt = times[1] - times[0]
        p_t0 = _p_from_slice(V_grid[0])
        p_t1 = _p_from_slice(V_grid[1])
        theta_g = (p_t1 - p_t0) / dt         # per year; per-day = theta_g/365
    else:
        theta_g = float("nan")               # degenerate grid

    # --- vega via central bump in sigma (be nice if sigma is tiny) ---
    dsig = min(max(1e-4, 0.5*sigma), abs(vol_bump))
    if sigma > dsig:
        p_up, _ = american_pde_price_banded(S0,K,T,sigma+dsig, r_func,q_func,dividends,cp,NS,NT,theta,rannacher,Smax,False)
        p_dn, _ = american_pde_price_banded(S0,K,T,sigma-dsig, r_func,q_func,dividends,cp,NS,NT,theta,rannacher,Smax,False)
        vega = (p_up - p_dn) / (2.0*dsig)
    else:  # forward diff fallback if sigma is too small
        p_up, _ = american_pde_price_banded(S0,K,T,sigma+dsig, r_func,q_func,dividends,cp,NS,NT,theta,rannacher,Smax,False)
        vega = (p_up - p0) / dsig

    # --- rho via parallel rate shift r(t) ± dr ---
    dr = abs(rate_bump)
    r_up = (lambda t, rf=r_func, bump=dr: rf(t) + bump)
    r_dn = (lambda t, rf=r_func, bump=dr: rf(t) - bump)
    p_r_up, _ = american_pde_price_banded(S0,K,T,sigma, r_up,q_func,dividends,cp,NS,NT,theta,rannacher,Smax,False)
    p_r_dn, _ = american_pde_price_banded(S0,K,T,sigma, r_dn,q_func,dividends,cp,NS,NT,theta,rannacher,Smax,False)
    rho = (p_r_up - p_r_dn) / (2.0*dr)

    out = {
        "price": p0,
        "delta": delta,
        "gamma": gamma,
        "vega": vega,          # per 1.00 (absolute) vol; per 1% vol = vega*0.01
        "theta": theta_g,      # per year; per day = theta_g/365
        "rho": rho             # per 1.00 change in rate; per bp = rho*1e-4
    }

    # pass grid back only if the caller asked for it
    if return_grid:
        out["grid_S"] = Sg
        out["grid_times"] = times
        out["grid_V"] = V_grid

    return out

In [9]:
import pandas as pd
import numpy as np

# --- helpers -------------------------------------------------------
def _to_dt(x): return pd.to_datetime(x, infer_datetime_format=True)
def _yearfrac(d0, d1): return (pd.Timestamp(d1) - pd.Timestamp(d0)).days / 365.0

def _vol_to_float(v):
    if isinstance(v, str):
        v = v.strip()
        if v.endswith("%"): return float(v[:-1]) / 100.0
        return float(v)
    v = float(v)
    return v/100.0 if v > 1.5 else v  # treat 36 => 0.36

def _const_fn(x): return (lambda t, _x=float(x): _x)

# --- main: take option + dividend DataFrames and return priced table ----------
def price_table_with_divs(
    options_df: pd.DataFrame,
    dividends_df: pd.DataFrame,
    valuation_date=None,
    r=0.05, q=0.0,                               # constants or callables
    NS=400, NT=800, scheme_theta=.5, rannacher=4, Smax=None
):
    """
    options_df columns (case-insensitive):
      mult, Expiration, Strike, Spot, CP, EA, Texp, Vol
      - CP: 'C' or 'P'
      - EA: 'A' (american) or 'E' (european)  [american assumed unless you add the small flag in PDE]
      - Texp (years) optional; if missing we use Expiration vs valuation_date.
      - Vol accepts 0.36, '36%', or 36
    dividends_df columns:
      Date, Amount   (ex-div cash)
    """
    val_date = pd.Timestamp.today().normalize() if valuation_date is None else _to_dt(valuation_date)

    # normalize column names
    opt = options_df.copy()
    opt.columns = [c.strip().lower() for c in opt.columns]
    divs = dividends_df.copy()
    divs.columns = [c.strip().lower() for c in divs.columns]
    if 'date' not in divs or 'amount' not in divs:
        raise ValueError("dividends_df must have columns: Date, Amount")

    # precompute dividend times (years from valuation date)
    divs['t_years'] = (_to_dt(divs['date']) - val_date).dt.days / 365.0

    # constant or callable r/q
    r_func = r if callable(r) else _const_fn(r)
    q_func = q if callable(q) else _const_fn(q)

    results = []

    for _, row in opt.iterrows():
        S0 = float(row['spot'])
        K  = float(row['strike'])
        borrow=float(row['borrow'])
        cp = 'C' if str(row.get('cp','C')).upper().startswith('C') else 'P'
        mult = float(row.get('mult', 100))

        # expiry in years
        if 'texp' in row and pd.notna(row['texp']):
            T = float(row['texp'])
        else:
            if 'expiration' not in row or pd.isna(row['expiration']):
                raise ValueError("Need either Texp or Expiration")
            T = _yearfrac(val_date, _to_dt(row['expiration']))

        sigma = _vol_to_float(row['vol'])
        q_func = q if callable(q) else _const_fn(q+borrow)
        #q_func = q_func + borrow
        #q_func = lambda t: q(t)+borrow if callable(q) else _const_fn(q+borrow)
        # dividends for THIS option (0 < t <= T)
        div_list = [(float(t), float(a)) for t, a in zip(divs['t_years'], divs['amount']) if 0.0 < t <= T]

        # --- call your PDE pricer (assumes you already defined it above) ----
        # american_pde_price_and_greeks: from your earlier code
        out = american_pde_price_and_greeks(
            S0, K, T, sigma,
            r_func=r_func, q_func=q_func,
            dividends=div_list, cp=cp,
            NS=NS, NT=NT, theta=scheme_theta, rannacher=rannacher, Smax=Smax,
            return_grid=False
        )

        # scale by contract multiplier; also give convenient 01s
        res = {
            'Price':     out['price'],
            'delta':     out['delta'],
            'gamma':     out['gamma'],
            'vega':      out['vega'],
            'theta':     out['theta'],
            'rho':       out['rho'],
            'Theo':      out['price'] * mult,
            'Delta':     out['delta'] * mult,
            'Gamma':     out['gamma'] * mult,                         # per $1
            'Vega':      out['vega']  * mult,                         # per 1.00 vol
            'Vega01':    out['vega']  * 0.01 * mult,                  # per 1 vol point
            'Theta_yr':  out['theta'] * mult,                          # per year
            'Theta_day': out['theta'] / 365.0 * mult,                 # per calendar day
            'Rho':       out['rho']   * mult,                         # per 1.00 rate
            'Rho01':     out['rho']   * 1e-4 * mult                   # per 1bp
        }

        results.append(res)

    res_df = pd.concat([opt.reset_index(drop=True), pd.DataFrame(results)], axis=1)
    return res_df

# Options Underlying

In [37]:
options_df = pd.DataFrame({
    'mult':[100,100],
    'Expiration': ['1/15/27','1/15/27'],
    'Strike':[250,250],
    'Spot':[252.29,252.29],
    'CP':['P','C'],
    'EA':['A','A'],
    'Texp':[1.2416,1.2416],
    'Vol':['27.55%','27.55%'],
    'Borrow':[0.01,0.01]
})

dividends_df = pd.DataFrame({
    'Date':['11/8/26','2/8/27','5/10/27','8/9/27','11/8/27','2/8/28','5/8/28','8/8/28'],
    'Amount':[0.0,0.0,0.0,0.0,0.40,0.40,0.40,0.40]
})

str_underlying='AAPL'
pd_val_date=pd.Timestamp('10/17/2025') 

In [14]:

priced_ql = price_table_with_divs_ql(
    options_df, dividends_df,
    #valuation_date='10/17/2025',
    valuation_date=pd_val_date.strftime('%m/%d/%Y'),
    r=0.0308,                  # use your carry here; bump if needed to match combo
    init_vol=0.2755,           # optional: seed for the SimpleQuote
    t_grid=400, x_grid=200,
    want_greeks=True,
    compute_combo=True
)

print(priced_ql)

   mult Expiration  Strike    Spot CP EA    Texp     Vol  Borrow       Theo  \
0   100    1/15/27     250  252.29  P  A  1.2416  27.55%    0.01  26.488635   
1   100    1/15/27     250  252.29  C  A  1.2416  27.55%    0.01  34.420027   

      Delta     Gamma      Theta      Vega  Combo_C_minus_P  
0 -0.404100  0.005221  -9.680966  1.075306         7.931392  
1  0.598174  0.004900 -13.920870  1.071006         7.931392  


# IVolatility Data

In [35]:
import ivolatility as ivol
import os
import dotenv

dotenv.load_dotenv()
api_key = os.getenv("IVOL_API_KEY")
if not api_key:
    raise RuntimeError("❌ IVOL_API_KEY not found in .env file or environment.")

ivol.setLoginParams(apiKey=api_key)
def get_ivol_yc(pd_date):
    getMarketData = ivol.setMethod('/equities/interest-rates')
    marketData = getMarketData(from_=pd_date.strftime('%Y-%m-%d'), till=pd_date.strftime('%Y-%m-%d'), currency='USD')
    return marketData

def apply_ivol_yc(marketData,df_options):
    df_results=df_options.copy()
    #df_results['yc'] = None
    for index, row in df_options.iterrows():
        period = int(row['Texp'] * 252)
        yc = marketData[marketData['period'] == period]['rate'].values[0]/100
        df_results.at[index, 'int_rate'] = yc
    return df_results

yc_results=get_ivol_yc(pd_val_date)

options_df2=apply_ivol_yc(yc_results,options_df)
options_df2




,mult,Expiration,Strike,Spot,CP,EA,Texp,Vol,Borrow,int_rate
0,100,1/15/27,250,252.29,P,A,1.2416,27.55%,0.01,0.034684
1,100,1/15/27,250,252.29,C,A,1.2416,27.55%,0.01,0.034684


In [36]:
# minimal exact-match resolver for iVolatility EOD chains
# depends on: pandas as pd, numpy as np, QuantLib not required here

import os, asyncio
import pandas as pd, numpy as np
from datetime import datetime

# --- use your existing helpers if you have them; duplicated here for self-containment ---
try:
    import ivolatility as ivol  # type: ignore
    _IVOL_SDK = True
except Exception:
    ivol = None
    _IVOL_SDK = False

def init_ivol_options_client():
    if not _IVOL_SDK:
        raise RuntimeError("ivolatility SDK not installed. `pip install ivolatility`")
    api_key = os.getenv("IVOL_API_KEY")
    if not api_key:
        raise RuntimeError("IVOL_API_KEY not found in environment.")
    ivol.setLoginParams(apiKey=api_key)
    return ivol.setMethod('/equities/eod/stock-opts-by-param')

def _numify(df: pd.DataFrame, col: str, dtype: str = "float64"):
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype(dtype)
    else:
        df[col] = pd.Series(pd.NA, index=df.index, dtype=dtype)

def _normalize_chain(df: pd.DataFrame) -> pd.DataFrame:
    """Light rename + types + computed mark."""
    if df is None or df.empty:
        return pd.DataFrame()
    colmap = {
        "expiration_date": "expiration",
        "call_put": "cp",
        "price_strike": "strike",
        "Bid": "bid",
        "Ask": "ask",
        "iv": "iv",
        "option_id": "optionId",
        "option_symbol": "osym",
        "openinterest": "openInterest",
        "volume": "volume",
        "underlying_price": "underlying_price",
        "delta": "delta",
        "gamma": "gamma",
        "vega": "vega",
    }
    df = df.rename(columns={k: v for k, v in colmap.items() if k in df.columns}).copy()
    if "cp" in df.columns:
        df["cp"] = df["cp"].astype(str).str.upper().str[0]
    for field in ["bid","ask","iv","strike","delta","gamma","vega","underlying_price"]:
        _numify(df, field)
    for field in ["openInterest","volume"]:
        _numify(df, field, dtype="Int64")
    df["mark"] = np.where(np.isfinite(df["bid"]) & np.isfinite(df["ask"]),
                          (df["bid"] + df["ask"]) / 2.0,
                          np.where(np.isfinite(df["bid"]), df["bid"], df["ask"]))
    if "expiration" in df.columns:
        df["expiration"] = pd.to_datetime(df["expiration"]).dt.strftime("%Y-%m-%d")
    return df

def _ensure_iso_date(v) -> str:
    """Accepts 'YYYY-MM-DD', 'M/D/YY', datetime, etc. -> 'YYYY-MM-DD'."""
    if isinstance(v, str):
        # try a couple formats
        for fmt in ("%Y-%m-%d", "%m/%d/%Y", "%m/%d/%y"):
            try:
                return datetime.strptime(v, fmt).strftime("%Y-%m-%d")
            except ValueError:
                continue
    if isinstance(v, (pd.Timestamp, datetime)):
        return pd.Timestamp(v).strftime("%Y-%m-%d")
    raise ValueError(f"Unrecognized date format: {v}")

def fetch_ivol_chain(symbol: str, trade_date: str, getMarketData=None,
                     dte_from: int = 0, dte_to: int = 1200,
                     mny_from: int = -200, mny_to: int = 200) -> pd.DataFrame:
    """Download C & P once and return a single normalized DataFrame."""
    if getMarketData is None:
        getMarketData = init_ivol_options_client()
    sym = symbol.upper()
    def _fetch(cp: str):
        return getMarketData(symbol=sym, tradeDate=trade_date,
                             dteFrom=dte_from, dteTo=dte_to,
                             moneynessFrom=mny_from, moneynessTo=mny_to, cp=cp)
    # SDK is sync; fetch serially for simplicity
    calls = _normalize_chain(_fetch('C'))
    puts  = _normalize_chain(_fetch('P'))
    if calls.empty and puts.empty:
        return pd.DataFrame()
    df = pd.concat([calls, puts], ignore_index=True)
    return df

def pick_by_maturity_and_strike(
    chain_df: pd.DataFrame,
    legs_df: pd.DataFrame,
    symbol: str,
    snap_to_nearest: bool = False
) -> dict:
    """
    legs_df must have columns: ['Expiration','Strike','CP'] (case-insensitive ok).
    Returns dict { ok, dataset, issues, matched_df }.
    """
    if chain_df.empty:
        return {"ok": False, "dataset": [], "issues": [{"code":"NO_CHAIN","msg":"empty chain"}], "matched_df": chain_df}

    # standardize leg column names
    cols = {c.lower(): c for c in legs_df.columns}
    need = {"expiration","strike","cp"}
    if not need.issubset(set(cols.keys())):
        raise ValueError("legs_df must have columns Expiration, Strike, CP")
    L = legs_df.rename(columns={
        cols["expiration"]: "Expiration",
        cols["strike"]: "Strike",
        cols["cp"]: "CP"
    }).copy()

    # normalize leg fields
    L["CP"] = L["CP"].astype(str).str.upper().str[0]
    L["Strike"] = pd.to_numeric(L["Strike"], errors="coerce").astype(float)
    L["ExpirationISO"] = L["Expiration"].apply(_ensure_iso_date)

    # fast lookup: (exp, cp) -> frame sorted by strike
    by_key = { (e, c): g.sort_values("strike").reset_index(drop=True)
               for (e, c), g in chain_df.groupby(["expiration","cp"], dropna=True) }

    issues, rows = [], []
    for i, leg in L.iterrows():
        exp = leg["ExpirationISO"]; cp = leg["CP"]; k = float(leg["Strike"])
        g = by_key.get((exp, cp))
        if g is None or g.empty:
            issues.append({"code":"NO_EXP_CP","msg":f"{symbol} {cp} @ {exp} not in chain"})
            continue
        # exact strike?
        exact = g[g["strike"] == k]
        if not exact.empty:
            sel = exact.iloc[0]
            snapped = False
        elif snap_to_nearest:
            # nearest strike fallback
            strikes = g["strike"].values
            idx = int(np.argmin(np.abs(strikes - k)))
            sel = g.iloc[idx]
            snapped = True
            issues.append({"code":"SNAP_STRIKE","msg":f"{symbol} {cp} {k} -> {float(sel['strike'])} @ {exp}"})
        else:
            issues.append({"code":"NO_STRIKE","msg":f"{symbol} {cp} {k} @ {exp} not listed"})
            continue

        rows.append({
            "symbol": symbol.upper(),
            "cp": cp,
            "expiration": exp,
            "strike": float(sel["strike"]),
            "snapped_strike": bool(snapped),
            "bid": float(sel.get("bid")) if pd.notna(sel.get("bid")) else np.nan,
            "ask": float(sel.get("ask")) if pd.notna(sel.get("ask")) else np.nan,
            "mark": float(sel.get("mark")) if pd.notna(sel.get("mark")) else np.nan,
            "iv": float(sel.get("iv")) if pd.notna(sel.get("iv")) else np.nan,
            "openInterest": sel.get("openInterest"),
            "volume": sel.get("volume"),
            "delta": float(sel.get("delta")) if pd.notna(sel.get("delta")) else np.nan,
            "gamma": float(sel.get("gamma")) if pd.notna(sel.get("gamma")) else np.nan,
            "vega": float(sel.get("vega")) if pd.notna(sel.get("vega")) else np.nan,
            "underlying_price": float(sel.get("underlying_price")) if pd.notna(sel.get("underlying_price")) else np.nan,
            "optionId": sel.get("optionId") or sel.get("osym"),
        })

    return {
        "ok": len(rows) == len(L),
        "dataset": rows,
        "issues": issues,
        "matched_df": pd.DataFrame(rows)
    }

# --- one-call convenience wrapper ---
def fetch_and_pick(symbol: str, trade_date: str, legs_df: pd.DataFrame, getMarketData=None, snap_to_nearest=False):
    chain = fetch_ivol_chain(symbol, trade_date, getMarketData=getMarketData)
    return pick_by_maturity_and_strike(chain, legs_df, symbol, snap_to_nearest=snap_to_nearest)

In [38]:
# legs_df with your desired contracts:
# columns: Expiration (any parseable date), Strike (float), CP ('C'/'P')
legs_df=options_df2[['Expiration','Strike','CP']]
out = fetch_and_pick(str_underlying, trade_date=pd_val_date.strftime('%Y-%m-%d'), legs_df=legs_df, snap_to_nearest=False)
print(out["issues"])     # any misses or snaps
print(out["matched_df"]) # exact rows you can pass to your pricer

[]
  symbol cp  expiration  strike  snapped_strike   bid    ask    mark  \
0   AAPL  P  2027-01-15   250.0           False  24.6  25.00  24.800   
1   AAPL  C  2027-01-15   250.0           False  36.7  37.65  37.175   

         iv  openInterest  volume     delta     gamma      vega  \
0  0.267427          5031      57 -0.401476  0.005546  1.071017   
1  0.287485          9767      57  0.618236  0.004756  1.068378   

   underlying_price   optionId  
0            252.29  134511704  
1            252.29  134511703  


In [49]:
type(options_df2['Expiration'][0])



str

In [51]:
type(out["matched_df"]["expiration"][0])


str

In [ ]:
out["matched_df"]
out_relevant=out["matched_df"][['expiration','strike','cp','bid','ask','iv','openInterest','volume','delta','gamma','vega','underlying_price','optionId']]
df_combine=pd.concat([options_df2,out_relevant],axis=1)
df_combine
df_show=

,mult,Expiration,Strike,Spot,CP,EA,Texp,Vol,Borrow,int_rate,...,bid,ask,iv,openInterest,volume,delta,gamma,vega,underlying_price,optionId
0,100,1/15/27,250,252.29,P,A,1.2416,27.55%,0.01,0.034684,...,24.6,25.00,0.267427,5031,57,-0.401476,0.005546,1.071017,252.29,134511704
1,100,1/15/27,250,252.29,C,A,1.2416,27.55%,0.01,0.034684,...,36.7,37.65,0.287485,9767,57,0.618236,0.004756,1.068378,252.29,134511703
